In [1]:
pip install Flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install flask-cors

Note: you may need to restart the kernel to use updated packages.


In [3]:
python -m http.server 8080

SyntaxError: invalid syntax (518402593.py, line 1)

In [ ]:
import numpy as np
import os
import json

In [ ]:
### CLASSES ###
class Cell:
    def __init__(self, input_val):
        self.value = 0
        if(input_val != '*'):
            self.value = int(input_val)


class Board:
    def __init__(self, file_name):
        # Get Boards folder
        current_directory = os.getcwd() #Current working directory
        parent_directory = os.path.dirname(current_directory)
        new_folder_path = os.path.join(parent_directory, 'Unsolved-Boards')
        text_file_path = os.path.join(new_folder_path, file_name)

        # Check if the folder exists, if not, create it
        if not os.path.exists(text_file_path):
            print("ERROR - the path " + text_file_path + " does not exist")
            return

        
        # Read File
        fileReader = open(text_file_path, "r")    
        file_as_text = fileReader.read()
        text_to_array = file_as_text.split('\n')
        text_to_array = np.array([[tile for tile in row] for row in text_to_array if len(row) > 0])
        #print(text_to_array)
        
        
        # Turn File into Board object
        self.board = []
        
        for row in text_to_array:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
        
    def print_board(self):
        for row in self.board:
            string = ""
            
            for cell in row:
                if cell.value == 0:
                    string += "*" + " "
                else:
                    string += str(cell.value) + " "
                
            print(string)
            
    def toJSON(self):
        json_data = {'board': []}
        for row in self.board:
            row_data = []
            for cell in row:
                row_data.append(cell.value)
            json_data['board'].append(row_data)
        return json.dumps(json_data)

In [ ]:
### BACK TRACKING ALGORTHM ###

#Creating the Constraint Satisfaction Problem variable:
sudoku_csp = {
    #The variable item contains a list of the cells of the Sudoku Board
    'variables': [(i, j) for i in range(9) for j in range(9)],
    
    #The domain item contains a dictionary of each variable containing a list of numbers from 1 to 9
    'domains': {variable: list(range(1, 10)) for variable in [(i, j) for i in range(9) for j in range(9)]},
    
    #The constraint item contains a dictionary of each constraint type
    #Each constraint hold a lamba function for checking if a given value violates the contraint or not
    'constraints': {
        'row': lambda board, row, col, num: num not in board[row],
        'column': lambda board, row, col, num: num not in [board[i][col] for i in range(9)],
        'subgrid': lambda board, row, col, num: all(num != board[i][j] for i in range(3*(row//3), 3*(row//3) + 3) for j in range(3*(col//3), 3*(col//3) + 3))
    }
}

In [ ]:
#The class for solving the Puzzle
def solve_sudoku_csp(board, csp):
    
    #Creating variables holding the different components of the CSP variable
    variables = csp['variables']
    domains = csp['domains']
    constraints = csp['constraints']
    
    #Creating a dictionary to store the assigned cells
    assigned_cells = {}  
    
    #the backtracking function
    def backtrack(board):
        
        #Calls the function to find an unassigned cell
        empty_cell = find_empty_cell(board)
        
        #if the empty cell returns None, the board is solved
        if not empty_cell:
            return True
        
        #set the row and column of the empty cell
        row, col = empty_cell
        
        #go through each value in the domain of the empty cell to find the correct value
        for num in domains[empty_cell]:
            
            #if the current value does not violate any constraint, assign the value to the empty cell
            if is_valid(board, row, col, num, constraints):
                board[row][col] = num
                
                #Record the assigned cell
                assigned_cells[(row, col)] = num
                
                #call the backtracking function for another empty cell
                #if there are no more empty cells, return true
                if backtrack(board):
                    return True
                #Backtrack and remove the cell
                board[row][col] = 0  
                del assigned_cells[(row, col)] 
                
        #No solution found
        return False  
    
    #the function to find an empty cell
    def find_empty_cell(board):
        
        #Go through each row and column to find a cell that contains 0
        for i in range(9):
            for j in range(9):
                if board[i][j] == 0:
                    return (i, j)
        return None
    
    #the function to check the board's consistency with the constraints
    def is_valid(board, row, col, num, constraints):
        
        #looping to go through each constraint and using the function inside to test for consistency
        for constraint_type, constraint_func in constraints.items():
            
            #if the function does not return true, then it is not consistent
            if not constraint_func(board, row, col, num):
                return False
        return True
    
    #if the backtracking function returns true, return the board
    if backtrack(board):
        return True, board, assigned_cells  # Return assigned cells along with the solution
    else:
        return False, None, None



In [ ]:
#function for copying the board into a list of lists of ints
def CopyBoard(board):
    newBoardList = []
    for row in board:
    
        row_list = [] 
        for cell in row:
            row_list.append(cell.value)
    
        newBoardList.append(row_list)
        
    return newBoardList   

In [ ]:
game1 = Board("Puzzle_1.txt")
game1.print_board()

In [ ]:
#Creating a new board to get the integer values of each row
unsolvedBoard = CopyBoard(game1.board)

unsolvedBoard

In [ ]:
solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)
if solved:
    print("Sudoku Solved Successfully!")
    for row in solution:
        print(row)
else:
    print("No solution exists!")

In [ ]:
game2 = Board("Puzzle_2.txt")
game2.print_board()

In [ ]:
unsolvedBoard = CopyBoard(game2.board)

unsolvedBoard

In [ ]:
solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)
if solved:
    print("Sudoku Solved Successfully!")
    for row in solution:
        print(row)
else:
    print("No solution exists!")

In [ ]:
game3 = Board("Puzzle_3.txt")
game3.print_board()

In [ ]:
unsolvedBoard = CopyBoard(game3.board)

unsolvedBoard

In [ ]:
solved, solution, assignments = solve_sudoku_csp(unsolvedBoard, sudoku_csp)
if solved:
    print("Sudoku Solved Successfully!")
    for row in solution:
        print(row)
else:
    print("No solution exists!")

In [ ]:
### SOLVING STRATEGIES ###



In [ ]:
### FRONT END CONNECTION ###
from flask import Flask, jsonify, request
from flask_cors import CORS
import pandas as pd
import numpy as np

def create_json():
    updates = [
        {
            "solve_type": "Solved cells",
            "eliminations": [{"x": 4, "y": 5, "values": [5]}],
            "found_value": 5,
            "group": None
        },
        {
            "solve_type": "Naked Single",
            "group": [{"x": 3, "y": 4, "values": [1, 2, 3, 5, 8]}],
            "eliminations": [{"x": 5, "y": 4, "values": [1, 9]}],
            "found_value": None
        },
        {
            "solve_type": "Hidden Single",
            "group": [{"x": 1, "y": 7, "values": [2, 3, 5, 6, 9]}],
            "eliminations": [{"x": 1, "y": 7, "values": [2, 6, 9]}],
            "found_value": {"x": 1, "y": 7, "value": 8}
        },
        {
            "solve_type": "Naked Pair",
            "group": [
                {"x": 2, "y": 6, "values": [2, 5]},
                {"x": 2, "y": 8, "values": [2, 5]}
            ],
            "eliminations": [
                {"x": 2, "y": 3, "values": [2, 5]},
                {"x": 2, "y": 5, "values": [2, 5]}
            ],
            "found_value": None
        }
    ]
    
    json_data = {"updates": updates}
    return json.dumps(json_data, indent=2)

app = Flask(__name__)
CORS(app)

@app.route('/solve', methods=['POST'])
def solve():
    # Process the POST request and return JSON response
    result = create_json()
    return jsonify(result)

@app.route('/load', methods=['POST'])
def load():
    # Retrieve the file name from the request data or URL parameters
    fileName = request.form.get('fileName')  # Assuming the file name is sent in a form field

    # Process the request to load data from the file (e.g., using Board(fileName))
    temp = Board(fileName)
    print(temp.toJSON())
    return temp.toJSON()

if __name__ == '__main__':
    app.run(debug=True, port=8080, use_reloader=False)

In [ ]:
### COMBINED SOLVING AND MAIN FUNCTION ###